In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import lmdb
import pickle
import numpy as np

In [3]:
os.chdir('/home/azeghost/git/Generative_Models') # adress to git dir
print(os.getcwd())
sep_local = os.sep

/home/azeghost/git/Generative_Models


## Download data prep for Eval_dataset the old method

In [4]:
#DATA_DOWN_PATH = '..'+sep_local+'..'+sep_local+'..'+sep_local+'data'
DATA_DOWN_PATH = os.getcwd() + sep_local+'data'
Script_dir = os.getcwd() + sep_local+'data'+sep_local+'download_gt_data.sh'
# Script call to download "dsprites_full" dataset_name 
dataset_name = 'small_norb'
!/bin/bash $Script_dir -f $DATA_DOWN_PATH -d $dataset_name

--2020-09-24 15:25:46--  https://cs.nyu.edu/~ylclab/data/norb-v1.0-small/smallnorb-5x01235x9x18x6x2x96x96-testing-cat.mat.gz
Resolving cs.nyu.edu (cs.nyu.edu)... 128.122.49.30
Connecting to cs.nyu.edu (cs.nyu.edu)|128.122.49.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347 [application/x-gzip]
Saving to: ‘/home/azeghost/git/Generative_Models/data/.gt_datasets/small_norb/smallnorb-5x01235x9x18x6x2x96x96-testing-cat.mat.gz’

/home/azeghost/git/ 100%[===================>]     347  --.-KB/s    in 0s      

2020-09-24 15:25:48 (20.4 MB/s) - ‘/home/azeghost/git/Generative_Models/data/.gt_datasets/small_norb/smallnorb-5x01235x9x18x6x2x96x96-testing-cat.mat.gz’ saved [347/347]

gzip: /home/azeghost/git/Generative_Models/data/.gt_datasets/small_norb/smallnorb-5x01235x9x18x6x2x96x96-testing-cat.mat already exists; do you wish to overwrite (y or n)? ^C


In [5]:
from data.gt_load.datasets import load
DATA_PATH =  os.getcwd() + sep_local+'data' +sep_local+'.gt_datasets'
absolute = os.path.abspath(DATA_PATH)
print("DATA_PATH",absolute)
# eval_dataset = load(dataset_name='dsprites_full', dataset_path=DATA_PATH)

DATA_PATH /home/azeghost/git/Generative_Models/data/.gt_datasets


In [6]:
def save_to_npz(datasetName: str, X, Y=None, L=None, Z=None):
#     print('Started Saving arrays to npz')
    BASE_PATH = os.path.join(os.getcwd()+os.sep + 'npz_data')
    if not os.path.exists(BASE_PATH):
        os.makedirs(BASE_PATH)
    file_name = "{0}.npz".format(datasetName)
    if os.path.exists(file_name):
        os.remove(file_name)
    np.savez(os.path.join(BASE_PATH, file_name), X=X, Y=Y, L=L, Z=Z)
#     print('Finished Saving arrays to npz')


In [7]:
def load_from_npz(BASE_PATH: str, datasetName: str):
#     print('Started loading arrays from npz')
#     BASE_PATH = os.path.join(os.getcwd()+os.sep + 'npz_data')
    file_name = "{0}.npz".format(datasetName)
    data = np.load(os.path.join(BASE_PATH, file_name))
#     print('Finished loading arrays from npz')
    if 'celeb_a' in datasetName:
        return data['X'], data['Y'], data['L']
    return data['X'], data['Y'], data['L']

In [ ]:
"""SmallNORB dataset."""
import os
from data.gt_load import gt_data, util
# import dask.array as da
import numpy as np
import PIL
from six.moves import range
import tensorflow as tf
from tqdm import tqdm
import math

class SmallNORB_2(gt_data.GroundTruthData):
    """SmallNORB dataset.

    The data set can be downloaded from
    https://cs.nyu.edu/~ylclab/data/norb-v1.0-small/. Images are resized to 64x64.

    The ground-truth factors of variation are:
    0 - category (5 different values)
    1 - elevation (9 different values)
    2 - azimuth (18 different values)
    3 - lighting condition (6 different values)

    The instance in each category is randomly sampled when generating the images.
    """

    def __init__(self, data_template, data_chuncks):
        self.images, features = _load_small_norb_chunks(data_template,
                                                        data_chuncks)
        latents_names = ['category', 'instance', 'elevation', 'azimuth', 'lighting']
        
        
        for idx, n in tqdm(enumerate(range(0,48600,10000)),  total=math.ceil(48600/10000)):
                latents_values = features[n:n+10000]
                imgs = self.images[n:n+10000]
                
                save_to_npz(datasetName = f'SmallNORB_{idx}', X = imgs, Y=latents_values, L = latents_names)
        

        
        
#         self.factor_sizes = [5, 10, 9, 18, 6]
#         # Instances are not part of the latents space.
#         self.latents_factor_indices = [0, 2, 3, 4]
#         self.num_total_factors = features.shape[1]
#         self.index = util.StateSpaceAtomIndex(self.factor_sizes, features)
#         self.state_space = util.SplitDiscreteStateSpace(self.factor_sizes,
#                                                         self.latents_factor_indices)

    @property
    def num_factors(self):
        return self.state_space.num_latents_factors

    @property
    def factors_num_values(self):
        return [self.factor_sizes[i] for i in self.latents_factor_indices]

    @property
    def observation_shape(self):
        return [64, 64, 1]

    def sample_factors(self, num, random_state):
        """Sample a batch of factors Y."""
        return self.state_space.sample_latents_factors(num, random_state)

    def sample_observations_from_factors(self, factors, random_state):
        all_factors = self.state_space.sample_all_factors(factors, random_state)
        indices = self.index.features_to_index(all_factors)
        return np.expand_dims(self.images[indices].astype(np.float32), axis=3)


def _load_small_norb_chunks(path_template, chunk_names):
    """Loads several chunks of the small norb data set for final use."""
    list_of_images, list_of_features = _load_chunks(path_template, chunk_names)
    features = np.concatenate(list_of_features, axis=0)
    features[:, 3] = features[:, 3] / 2  # azimuth values are 0, 2, 4, ..., 24
    return np.concatenate(list_of_images, axis=0), features


def _load_chunks(path_template, chunk_names):
    """Loads several chunks of the small norb data set into lists."""
    list_of_images = []
    list_of_features = []
    for chunk_name in chunk_names:
        norb = _read_binary_matrix(path_template.format(chunk_name, "dat"))
        list_of_images.append(_resize_images(norb[:, 0]))
        norb_class = _read_binary_matrix(path_template.format(chunk_name, "cat"))
        norb_info = _read_binary_matrix(path_template.format(chunk_name, "info"))
        list_of_features.append(np.column_stack((norb_class, norb_info)))
    return list_of_images, list_of_features


def _read_binary_matrix(filename):
    """Reads and returns binary formatted matrix stored in filename."""
    with tf.io.gfile.GFile(filename, "rb") as f:
        s = f.read()
        magic = int(np.frombuffer(s, "int32", 1))
        ndim = int(np.frombuffer(s, "int32", 1, 4))
        eff_dim = max(3, ndim)
        raw_dims = np.frombuffer(s, "int32", eff_dim, 8)
        dims = []
        for i in range(0, ndim):
          dims.append(raw_dims[i])

        dtype_map = {
            507333717: "int8",
            507333716: "int32",
            507333713: "float",
            507333715: "double"
        }
        data = np.frombuffer(s, dtype_map[magic], offset=8 + eff_dim * 4)
    data = data.reshape(tuple(dims))
    return data


def _resize_images(integer_images):
    resized_images = np.zeros((integer_images.shape[0], 64, 64))
    for i in range(integer_images.shape[0]):
        image = PIL.Image.fromarray(integer_images[i, :, :])
        image = image.resize((64, 64), PIL.Image.ANTIALIAS)
        resized_images[i, :, :] = image
    return resized_images / 255.


In [8]:
def small_norb(dataset_path):
    SMALLNORB_TEMPLATE = os.path.join(
        dataset_path,
        "small_norb",
        "smallnorb-{}-{}.mat"
    )

    SMALLNORB_CHUNKS = [
         "5x46789x9x18x6x2x96x96-training",
         "5x01235x9x18x6x2x96x96-testing",
     ]
    return SmallNORB_2(data_template=SMALLNORB_TEMPLATE, data_chuncks=SMALLNORB_CHUNKS)

In [ ]:
dataset_path = '/home/azeghost/git/Generative_Models/data/.gt_datasets'
test_sm=small_norb(dataset_path)


In [9]:
import os
os.chdir('/home/azeghost/git/lmdb_new/LMDB_Datasets')
os.getcwd()
BASE_PATH = os.path.join('/home/azeghost/git/Generative_Models'+os.sep + 'npz_data')

In [10]:
from transformation.lmdb_transformer import LmdbTransformer
from tqdm import tqdm
import math

In [11]:
validation_percentage = 30
valid_format = 'png'
numpy_transformer = LmdbTransformer( validation_percentage, valid_format)

In [12]:
for idx, n in enumerate(range(0,48600,10000)):
    imgs, latents_values, latents_names = load_from_npz(BASE_PATH, datasetName = f'SmallNORB_{idx}')

    numpy_transformer.transform_store_from_numpy(images =imgs,
                                                 labels_values =latents_values , 
                                                 labels_names = latents_names,
                                                 lmdb_dir='.data/SmallNORB_LMDB',
                                                 category='full',
                                                 total_number_imgs = 48600,
                                                 file_idx = idx)
    


100%|██████████| 8600/8600 [00:22<00:00, 386.73it/s]


In [13]:
import tensorflow_datasets as tfds

builder = tfds.builder('smallnorb')
info = builder.info
# print(info)
meta1_info_dict = {}
meta1_info_dict .update(info.features)
meta1_info_dict.update({'name': info.name})
meta1_info_dict.update({'description':str( info.description)})
meta1_info_dict


INFO:absl:Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: smallnorb/2.0.0
INFO:absl:Load dataset info from /tmp/tmpx017mea9tfds
INFO:absl:Field info.description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.citation from disk and from code do not match. Keeping the one from code.


{'image': Image(shape=(96, 96, 1), dtype=tf.uint8),
 'image2': Image(shape=(96, 96, 1), dtype=tf.uint8),
 'label_category': ClassLabel(shape=(), dtype=tf.int64, num_classes=5),
 'instance': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
 'label_elevation': ClassLabel(shape=(), dtype=tf.int64, num_classes=9),
 'label_azimuth': ClassLabel(shape=(), dtype=tf.int64, num_classes=18),
 'label_lighting': ClassLabel(shape=(), dtype=tf.int64, num_classes=6),
 'name': 'smallnorb',
 'description': '\\\nThis database is intended for experiments in 3D object recognition from shape. It contains images of 50 toys belonging to 5 generic categories: four-legged animals, human figures, airplanes, trucks, and cars. The objects were imaged by two cameras under 6 lighting conditions, 9 elevations (30 to 70 degrees every 5 degrees), and 18 azimuths (0 to 340 every 20 degrees).\n\nThe training set is composed of 5 instances of each category (instances 4, 6, 7, 8 and 9), and the test set of the remaini

In [14]:
numpy_transformer.save_metadata('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/SmallNORB_LMDB'
                                ,meta1_info_dict)

TypeError: Object of type Image is not JSON serializable

In [8]:
numpy_transformer.get_metadata('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/SmallNORB_LMDB')

FileNotFoundError: [Errno 2] No such file or directory: '/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/SmallNORB_LMDB/meta_info.json'

# Storing from small npz to LMDB

# Read the lmdb 

#Todo
 - Env put outside of loops for write and read
 - Try the Dynamic code again with clean kernel 
 - Create Standartize as an option

In [ ]:
import lmdb
import pickle

def read_many_lmdb(lmdb_dir, num_images):

    images, labels = [], {}
    env = lmdb.open(lmdb_dir, readonly=True)

    # Start a new read transaction
    with env.begin() as txn:
        # Read all images in one single transaction, with one lock
        # We could split this up into multiple transactions if needed
        for image_id in range(num_images):
            data = txn.get(f"{image_id:08}".encode("ascii"))
            # Remember that it's a CIFAR_Image object 
            # that is stored as the value
            dataset = pickle.loads(data)
            images.append(dataset.get_image())
            
            labels_list = [attr for attr in dir(dataset) if not callable(getattr(dataset, attr)) and (not attr.startswith("__")) and 
                           (not attr in ['image','channels',  'size'] )]
            
            for label in labels_list:
                if label in labels.keys():
                    labels[label].append(eval(f'dataset.{label}'))
                else:
                    labels.update({label : [eval(f'dataset.{label}') ]})
    
    env.close()
    return {'images': images, **labels}

In [ ]:
ds = read_many_lmdb('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data/Dstripes_LMDB/_full', 35000)

In [ ]:
ds.keys()

In [ ]:
len(ds['images'])

In [ ]:
len(ds['color_value'])

In [ ]:
os.chdir('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data')

# Zip and upload to git

In [ ]:
!tar -cvjf dsprites.tar.bz Dstripes_LMDB/*

In [ ]:
!split -b 50M dsprites.tar.bz "dsprites.tar.part"
#split -b <max size> <name of zip or dir to zip/name> <split file name beginning>

In [ ]:
#Check if they are created
!ls -lh dsprites.tar.part*

In [ ]:
import os
os.chdir('/home/azeghost/git/lmdb_new/LMDB_Datasets/.data')

In [ ]:
!ls -la

In [ ]:
#move to correct folder and push to git
# !mkdir /home/azeghost/git/LMDB_Datasets/pokemon
# !mv pokemon.tar.part* /home/azeghost/git/Generative_Models/data/.pokemon
# !ls -la /home/azeghost/git/Generative_Models/data/.pokemon 

# Combine Them

In [ ]:
#pokemon_training
!cat pokemon.tar.part* > pokemon_combined.tar.bz
#!cat <split files put * at the end> > <final zip name>
!ls -la 
!pwd

In [ ]:
!mkdir ./.test

In [ ]:
!tar -xf pokemon_combined.tar.bz --directory ./.test

In [ ]:
!ls -la ./.test/Pokemon_LMDB

In [ ]:
# !rm -rf ./.test